In [1]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

## s3 bucket name --> yelp_data_parquet , folder--> parquet [store file from kaggle]

## s3 bucket name --> eda_parquet , folder--> parquet  [storing file after doing EDA]


# Reading Business file

In [ ]:
business = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .load("s3://yelp_data_parquet/parquet/business.parquet")

In [ ]:
business.cache()

In [ ]:
business1 = business.na.fill(value='Edinburgh',subset=["city"])

In [ ]:
business2 = business1.na.fill(value='',subset=["neighborhood"])

In [ ]:
business3 = business2.na.fill(value='NV',subset=["State"])

In [ ]:
business4 = business3.na.fill(value='',subset=["postal_code"])

In [ ]:
business5 = business4.na.fill(value=43.651070 , subset=["Latitude"])

In [ ]:
business6 = business5.na.fill(value=-79.347015 , subset=["longitude"])

## saving file

# Reading Review

In [ ]:
review_schema = StructType([
    StructField("review_id", StringType()),
    StructField("user_id", StringType()),
    StructField("business_id", StringType()),
    StructField("stars", FloatType()),
    StructField("date", StringType()),
    StructField("text", StringType()),
    StructField("useful", IntegerType()),
    StructField("funny", IntegerType()),
    StructField("cool", IntegerType())
])

In [ ]:
review= spark.read.format("parquet") \
                         .option("header", "true") \
                         .schema(review_schema) \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("dateFormat", "yyyy-MM-dd HH:mm:ss")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/review.parquet")

## saving file

# Reading Checkin

In [ ]:
checkin = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema","true")\
                         .load("s3://yelp_data_parquet/parquet/checkin.parquet")

## saving file

## Reading Tip

In [ ]:
tip_schema = StructType([
    StructField("text", StringType()),
    StructField("date", StringType()),
    StructField("likes", IntegerType()),
    StructField("business_id", StringType()),
    StructField("user_id", StringType()),
])

In [ ]:
tip = spark.read.format("parquet") \
                         .option("header", "true") \
                         .schema(tip_schema)\
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/tip.parquet")

In [ ]:
tip1 = tip.na.fill(value='',subset=["text"])

## saving file

## Reading Hours

In [ ]:
hours = spark.read.parquet("s3://yelp_data_parquet/parquet/hours.parquet",header=True)

In [ ]:
from pyspark.sql.functions import regexp_replace
hours1 = hours.withColumn('monday', regexp_replace('monday', 'None', ''))
hours2 = hours1.withColumn('tuesday', regexp_replace('tuesday', 'None', ''))
hours3 = hours2.withColumn('wednesday', regexp_replace('wednesday', 'None', ''))
hours4 = hours3.withColumn('thursday', regexp_replace('thursday', 'None', ''))
hours5 = hours4.withColumn('friday', regexp_replace('friday', 'None', ''))
hours6 = hours5.withColumn('saturday', regexp_replace('saturday', 'None', ''))
hours7 = hours6.withColumn('sunday', regexp_replace('sunday', 'None', ''))

## saving file

# Reading user

In [ ]:
user = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .load("s3://yelp_data_parquet/parquet/user.parquet")

In [ ]:
user.cache()

In [ ]:
user2 = user1.na.fill(value='Unknown',subset=["name"])


In [ ]:
user2 = user1.na.fill(value='Unknown',subset=["name"])

In [ ]:
user3 = user2.withColumn('name', regexp_replace('name', 'None', 'Unknown'))

In [ ]:
user4 = user3.withColumn('elite', regexp_replace('elite', 'None', ''))

In [ ]:
user5 = user4.withColumn('friends', regexp_replace('friends', 'None', ''))

In [ ]:
user6=user5.drop('fans','compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos')


In [ ]:
from pyspark.sql.functions import split

In [ ]:
user7 = user6.withColumn("Yelping_date", split(col("Yelping_since"), " ").getItem(0)).withColumn("col2", split(col("Yelping_since"), " ").getItem(1))

In [ ]:
user8=user7.drop('Yelping_since','col2')

In [ ]:
user9= user8.withColumnRenamed('Yelping_date', 'Yelping_since')

In [ ]:
user10 = user9.select('user_id','name','review_count','Yelping_since','friends','useful','funny','cool','elite','average_stars')

## saving file

# Reading attribute

In [ ]:
attribute = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/attribute.parquet")

In [ ]:
attribute.cache()

In [ ]:
attribute_1=attribute.drop('DietaryRestrictions_kosher', 'DietaryRestrictions_halal','DietaryRestrictions_soy-free',
'DietaryRestrictions_vegetarian','RestaurantsCounterService','DietaryRestrictions_gluten-free',
'DietaryRestrictions_dairy-free','DietaryRestrictions_vegan', 'BYOBCorkage', 'BYOB','CoatCheck',
'Ambience_hipster','Ambience_divey','Ambience_intimate','Ambience_trendy','Ambience_upscale')

In [ ]:
attribute_2 = attribute_1.withColumn('AcceptsInsurance', regexp_replace('AcceptsInsurance', 'Na', ''))\
                         .withColumn('ByAppointmentOnly', regexp_replace('ByAppointmentOnly', 'Na', ''))\
                         .withColumn('BusinessAcceptsCreditCards', regexp_replace('BusinessAcceptsCreditCards', 'Na', ''))\
                         .withColumn('BusinessParking_garage', regexp_replace('BusinessParking_garage', 'Na', ''))\
                         .withColumn('BusinessParking_street', regexp_replace('BusinessParking_street', 'Na', ''))\
                         .withColumn('BusinessParking_validated', regexp_replace('BusinessParking_validated', 'Na', ''))\
                         .withColumn('BusinessParking_lot', regexp_replace('BusinessParking_lot', 'Na', ''))\
                         .withColumn('BusinessParking_valet', regexp_replace('BusinessParking_valet', 'Na', ''))\
                         .withColumn('HairSpecializesIn_coloring', regexp_replace('HairSpecializesIn_coloring', 'Na', ''))\
                         .withColumn('HairSpecializesIn_africanamerican', regexp_replace('HairSpecializesIn_africanamerican', 'Na', ''))\
                         .withColumn('HairSpecializesIn_curly', regexp_replace('HairSpecializesIn_curly', 'Na', ''))\
                         .withColumn('HairSpecializesIn_perms', regexp_replace('HairSpecializesIn_perms', 'Na', ''))\
                         .withColumn('HairSpecializesIn_kids', regexp_replace('HairSpecializesIn_kids', 'Na', ''))\
                         .withColumn('HairSpecializesIn_extensions', regexp_replace('HairSpecializesIn_extensions', 'Na', ''))\
                         .withColumn('HairSpecializesIn_asian', regexp_replace('HairSpecializesIn_asian', 'Na', ''))\
                         .withColumn('HairSpecializesIn_straightperms', regexp_replace('HairSpecializesIn_straightperms', 'Na', ''))\
                         .withColumn('RestaurantsPriceRange2', regexp_replace('RestaurantsPriceRange2', 'Na', ''))\
                         .withColumn('GoodForKids', regexp_replace('GoodForKids', 'Na', ''))\
                         .withColumn('WheelchairAccessible', regexp_replace('WheelchairAccessible', 'Na', ''))\
                         .withColumn('BikeParking', regexp_replace('BikeParking', 'Na', ''))\
                         .withColumn('Alcohol', regexp_replace('Alcohol', 'Na', ''))\
                         .withColumn('HasTV', regexp_replace('HasTV', 'Na', ''))\
                         .withColumn('NoiseLevel', regexp_replace('NoiseLevel', 'Na', ''))\
                         .withColumn('RestaurantsAttire', regexp_replace('RestaurantsAttire', 'Na', ''))\
                         .withColumn('Music_dj', regexp_replace('Music_dj', 'Na', ''))\
                         .withColumn('Music_background_music', regexp_replace('Music_background_music', 'Na', ''))\
                         .withColumn('Music_no_music', regexp_replace('Music_no_music', 'Na', ''))\
                         .withColumn('Music_karaoke', regexp_replace('Music_karaoke', 'Na', ''))\
                         .withColumn('Music_live', regexp_replace('Music_live', 'Na', ''))\
                         .withColumn('Music_video', regexp_replace('Music_video', 'Na', ''))\
                         .withColumn('Music_jukebox', regexp_replace('Music_jukebox', 'Na', ''))\
                         .withColumn('Ambience_romantic', regexp_replace('Ambience_romantic', 'Na', ''))\
                         .withColumn('Ambience_classy', regexp_replace('Ambience_classy', 'Na', ''))\
                         .withColumn('Ambience_touristy', regexp_replace('Ambience_touristy', 'Na', ''))\
                         .withColumn('Ambience_casual', regexp_replace('Ambience_casual', 'Na', ''))\
                         .withColumn('RestaurantsGoodForGroups', regexp_replace('RestaurantsGoodForGroups', 'Na', ''))\
                         .withColumn('Caters', regexp_replace('Caters', 'Na', ''))\
                         .withColumn('WiFi', regexp_replace('WiFi', 'Na', ''))\
                         .withColumn('RestaurantsReservations', regexp_replace('RestaurantsReservations', 'Na', ''))\
                         .withColumn('RestaurantsTakeOut', regexp_replace('RestaurantsTakeOut', 'Na', ''))\
                         .withColumn('HappyHour', regexp_replace('HappyHour', 'Na', ''))\
                         .withColumn('GoodForDancing', regexp_replace('GoodForDancing', 'Na', ''))\
                         .withColumn('RestaurantsTableService', regexp_replace('RestaurantsTableService', 'Na', ''))\
                         .withColumn('OutdoorSeating', regexp_replace('OutdoorSeating', 'Na', ''))\
                         .withColumn('RestaurantsDelivery', regexp_replace('RestaurantsDelivery', 'Na', ''))\
                         .withColumn('BestNights_monday', regexp_replace('BestNights_monday', 'Na', ''))\
                         .withColumn('BestNights_tuesday', regexp_replace('BestNights_tuesday', 'Na', ''))\
                         .withColumn('BestNights_friday', regexp_replace('BestNights_friday', 'Na', ''))\
                         .withColumn('BestNights_wednesday', regexp_replace('BestNights_wednesday', 'Na', ''))\
                         .withColumn('BestNights_thursday', regexp_replace('BestNights_thursday', 'Na', ''))\
                         .withColumn('BestNights_sunday', regexp_replace('BestNights_sunday', 'Na', ''))\
                         .withColumn('BestNights_saturday', regexp_replace('BestNights_saturday', 'Na', ''))\
                         .withColumn('GoodForMeal_dessert', regexp_replace('GoodForMeal_dessert', 'Na', ''))\
                         .withColumn('GoodForMeal_latenight', regexp_replace('GoodForMeal_latenight', 'Na', ''))\
                         .withColumn('GoodForMeal_lunch', regexp_replace('GoodForMeal_lunch', 'Na', ''))\
                         .withColumn('GoodForMeal_dinner', regexp_replace('GoodForMeal_dinner', 'Na', ''))\
                         .withColumn('GoodForMeal_breakfast', regexp_replace('GoodForMeal_breakfast', 'Na', ''))\
                         .withColumn('GoodForMeal_brunch', regexp_replace('GoodForMeal_brunch', 'Na', ''))\
                         .withColumn('Smoking', regexp_replace('Smoking', 'Na', ''))\
                         .withColumn('DriveThru', regexp_replace('DriveThru', 'Na', ''))\
                         .withColumn('DogsAllowed', regexp_replace('DogsAllowed', 'Na', ''))\
                         .withColumn('BusinessAcceptsBitcoin', regexp_replace('BusinessAcceptsBitcoin', 'Na', ''))\
                         .withColumn('Open24Hours', regexp_replace('Open24Hours', 'Na', ''))\
                         .withColumn('Corkage', regexp_replace('Corkage', 'Na', ''))\
                         .withColumn('AgesAllowed', regexp_replace('AgesAllowed', 'Na', ''))



## saving file